This notebook is used for: <br>
1) testing ARPA API functionalities <br>
2) testing functions to be implemented in the plugin <br>
3) testing libraries to be used and evaluate performances <br>

To mount you Google Drive Folder:

It is possible to manage this using Google Colab environmental file. You just need to install colab_env library, and use the following commands. The environmental variable will be stored in a vars.env file that is available in your MyDrive folder.

In [3]:
arpa_token = "riTLzYVRVdDaQtUkxDDaHRgJi" 

In [4]:
print(arpa_token)

riTLzYVRVdDaQtUkxDDaHRgJi


In [5]:
from sodapy import Socrata
import pandas as pd

In [6]:
stationsId = "nf78-nj6b" # Select meteo stations dataset containing positions and information about sensors

In [7]:
def connect_ARPA_api(token):
    """
    Function to connect to ARPA API.
    Parameters:
    - token: the ARPA token obtained from Open Data Lombardia website

    Returns:
    - client: client session
    """
    client = Socrata("www.dati.lombardia.it", app_token = token)

    return client

client = connect_ARPA_api(arpa_token)
stations_info = client.get_all(stationsId)

In [8]:
stations_df = pd.DataFrame(stations_info)
stations_df

,idsensore,tipologia,unit_dimisura,idstazione,nomestazione,quota,provincia,datastart,storico,cgb_nord,cgb_est,lng,lat,location,:@computed_region_6hky_swhk,:@computed_region_ttgh_9sm5,datastop
0,10373,Precipitazione,mm,687,Ferno v.Di Dio,215,VA,2007-08-13T00:00:00.000,N,5051773,481053,8.756970445453431,45.61924377994763,"{'latitude': '45.61924377994763', 'longitude':...",1,1,NaN
1,10376,Precipitazione,mm,706,Lecco v.Sora,272,LC,2008-07-22T00:00:00.000,N,5078987,531045,9.399950344681852,45.86374884127965,"{'latitude': '45.86374884127965', 'longitude':...",10,10,NaN
2,10377,Temperatura,°C,706,Lecco v.Sora,272,LC,2008-07-22T00:00:00.000,N,5078987,531045,9.399950344681852,45.86374884127965,"{'latitude': '45.86374884127965', 'longitude':...",10,10,NaN
3,10381,Umidità Relativa,%,706,Lecco v.Sora,272,LC,2008-07-22T00:00:00.000,N,5078987,531045,9.399950344681852,45.86374884127965,"{'latitude': '45.86374884127965', 'longitude':...",10,10,NaN
4,10382,Radiazione Globale,W/m²,706,Lecco v.Sora,272,LC,2008-07-31T00:00:00.000,N,5078987,531045,9.399950344681852,45.86374884127965,"{'latitude': '45.86374884127965', 'longitude':...",10,10,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227,9869,Umidità Relativa,%,672,Cornale v.Libertà,74,PV,2005-07-28T00:00:00.000,N,4987406,493238,8.914144599002409,45.04007657202963,"{'latitude': '45.04007657202963', 'longitude':...",7,7,NaN
1228,9933,Precipitazione,mm,677,Cremona Via Fatebenefratelli,43,CR,2006-04-10T00:00:00.000,N,4999315,582066,10.043836158369393,45.14254063221695,"{'latitude': '45.14254063221695', 'longitude':...",8,8,NaN
1229,9935,Radiazione Globale,W/m²,677,Cremona Via Fatebenefratelli,43,CR,2006-04-10T00:00:00.000,N,4999315,582066,10.043836158369393,45.14254063221695,"{'latitude': '45.14254063221695', 'longitude':...",8,8,NaN
1230,9938,Temperatura,°C,677,Cremona Via Fatebenefratelli,43,CR,2006-04-10T00:00:00.000,N,4999315,582066,10.043836158369393,45.14254063221695,"{'latitude': '45.14254063221695', 'longitude':...",8,8,NaN


Consideration: inside the plugin the data type for each columns must be set accordingly to QGIS data types.

**Request data time series**

In [22]:
def req_ARPA_data_sensor(client,id_sensor,start_date,end_date):
    """
    Function to request data from a single ARPA sensor using a query.
    Parameters:
    client: the client session
    id_sensor: the id of the sensor of interest
    start date (str): the start date in yyy-mm-dd format
    end date (str): the end date in yyy-mm-dd format

    Returns: time_series: time series of values requested with the query for the selected sensor
    """
    query = """
      select
          *
      where data between \'{}\' and \'{}\' and idsensore=\'{}\' 
      """.format(start_date,end_date, id_sensor)

    time_series = client.get("647i-nhxk",query=query) 
      
    return time_series

In [24]:
sensor_time_series = req_ARPA_data_sensor(client, 19165, "2023-01-15", "2023-01-16")
sensor_time_series

[{'idsensore': '19165',
  'data': '2023-01-15T20:00:00.000',
  'valore': '2.4',
  'idoperatore': '3',
  'stato': 'VA'},
 {'idsensore': '19165',
  'data': '2023-01-15T12:00:00.000',
  'valore': '1.9',
  'idoperatore': '3',
  'stato': 'VA'},
 {'idsensore': '19165',
  'data': '2023-01-16T00:00:00.000',
  'valore': '1',
  'idoperatore': '1',
  'stato': 'VA'},
 {'idsensore': '19165',
  'data': '2023-01-15T23:00:00.000',
  'valore': '2.5',
  'idoperatore': '3',
  'stato': 'VA'},
 {'idsensore': '19165',
  'data': '2023-01-15T09:00:00.000',
  'valore': '4.9',
  'idoperatore': '3',
  'stato': 'VA'},
 {'idsensore': '19165',
  'data': '2023-01-15T21:00:00.000',
  'valore': '1.1',
  'idoperatore': '1',
  'stato': 'VA'},
 {'idsensore': '19165',
  'data': '2023-01-15T07:00:00.000',
  'valore': '0.2',
  'idoperatore': '1',
  'stato': 'VA'},
 {'idsensore': '19165',
  'data': '2023-01-15T23:00:00.000',
  'valore': '1.1',
  'idoperatore': '1',
  'stato': 'VA'},
 {'idsensore': '19165',
  'data': '2023-01

In [25]:
sensor_df = pd.DataFrame(sensor_time_series)
sensor_df['data'] = pd.to_datetime(sensor_df['data'])
sensor_df = sensor_df.sort_values(by='data', ascending=False).reset_index(drop=True)
print(sensor_df)

   idsensore                data valore idoperatore stato
0      19165 2023-01-16 00:00:00      1           1    VA
1      19165 2023-01-16 00:00:00    2.8           3    VA
2      19165 2023-01-15 23:00:00    2.5           3    VA
3      19165 2023-01-15 23:00:00    1.1           1    VA
4      19165 2023-01-15 22:00:00      1           1    VA
5      19165 2023-01-15 22:00:00    2.7           3    VA
6      19165 2023-01-15 21:00:00    1.1           1    VA
7      19165 2023-01-15 21:00:00    2.3           3    VA
8      19165 2023-01-15 20:00:00    0.5           1    VA
9      19165 2023-01-15 20:00:00    2.4           3    VA
10     19165 2023-01-15 19:00:00    0.2           1    VA
11     19165 2023-01-15 19:00:00    1.7           3    VA
12     19165 2023-01-15 18:00:00    1.6           3    VA
13     19165 2023-01-15 18:00:00    0.1           1    VA
14     19165 2023-01-15 17:00:00    1.2           3    VA
15     19165 2023-01-15 17:00:00    0.1           1    VA
16     19165 2

In [ ]:
def meteo_sensor(year):
    """
    Function for selecting the correct link for downloading zipped .csv meteorological data from ARPA sensors.
    The meteorological data are available from the API during the current month.
    For older data it is necessary to download the .csv file containing the time series of the meteorological sensors.

            Parameters:
            year: the selected year for downloading the .csv file containing the meteorological sensors time series

            Returns:
            switcher.get: get request to retrieved the zip file containing the .csv  
    """
    switcher = {
        '2022': "https://www.dati.lombardia.it/download/mvvc-nmzv/application%2Fzip",
        '2021': "https://www.dati.lombardia.it/download/49n9-866s/application%2Fzip",
        '2020': "https://www.dati.lombardia.it/download/erjn-istm/application%2Fzip",
        '2019': "https://www.dati.lombardia.it/download/wrhf-6ztd/application%2Fzip",
        '2018': "https://www.dati.lombardia.it/download/sfbe-yqe8/application%2Fzip",
        '2017': "https://www.dati.lombardia.it/download/vx6g-atiu/application%2Fzip"
    }
    return switcher.get(year, "Invalid year. For current year data use the API request.")

In [ ]:
meteo_sensor()

In [27]:
import requests